In [3]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'final_project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [5]:
from house.models import Officetels, OneTwoRoom, Villa, Speed, Cafe, Convenience, Culture, Daiso, Fastfood, Health, Hospital, Laundry, Market


In [ ]:
solution('officetels', '전세','10000','0','0.24069938698647905',['37.5901684571627', '127.05156387144'])

In [19]:
input_rent = '전세'
input_deposit = '10000'
input_pay = 0
job = ['37.5901684571627', '127.05156387144']
con_0 = 'fast_num'
con_1 = 'mart_num'
con_2 = 'cafe_num'
gu = '0.24069938698647905'
table = 'Officetels'
stand_0 = 2
stand_1 = 2
stand_2 = 3
aa = table

In [119]:
    import psycopg2
    import requests
    from urllib.parse import urlparse

In [131]:
def latlon(address):
    conn_string = "host='localhost' dbname ='house' user='postgres' password='1111'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    gu = address.split(' ')[1]
    cur.execute(f"SELECT speed FROM SPEED \
            WHERE area='{gu}'") #직장이 속한 구의 대중교통 km/분 평균속도

    gu_speed = cur.fetchall()[0][0] 

    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 71dc765801a475a20aec8d82f539dd62"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['y'], document['x']]
        except:
            val = [0, 0]
    return gu_speed, val # 직장의 위치, 직장의 위도 경도

        
def score(input_house, input_rent,input_deposit,input_con,gu, job,input_pay,table, data):
    print('score')
    num = len(input_con)
    for i in range(num): # 편의시설의 개수만큼 for문 진행 
        if input_con[i] == '병원':
            globals()['con_{}'.format(i)] = 'hos_num'
            globals()['stand_{}'.format(i)] = 5
            globals()['table_con_{}'.format(i)] = 'hospital'
        elif input_con[i] == '마트':
            globals()['con_{}'.format(i)] = 'mart_num'
            globals()['stand_{}'.format(i)] = 2
            globals()['table_con_{}'.format(i)] = 'market'
        elif input_con[i] == '패스트푸드':
            globals()['con_{}'.format(i)] = 'fast_num'
            globals()['stand_{}'.format(i)] = 2
            globals()['table_con_{}'.format(i)] = 'fastfood'
        elif input_con[i] == '카페':
            globals()['con_{}'.format(i)] = 'cafe_num'
            globals()['stand_{}'.format(i)] = 3
            globals()['table_con_{}'.format(i)] = 'cafe'
        elif input_con[i] == '문화시설':
            globals()['con_{}'.format(i)] = 'cul_num'
            globals()['stand_{}'.format(i)] = 5
            globals()['table_con_{}'.format(i)] = 'culture'
        elif input_con[i] == '편의점':
            globals()['con_{}'.format(i)] = 'con_num'
            globals()['stand_{}'.format(i)] = 10
            globals()['table_con_{}'.format(i)] = 'convenience'
        elif input_con[i] == '세탁소':
            globals()['con_{}'.format(i)] = 'laun_num'
            globals()['stand_{}'.format(i)] = 7
            globals()['table_con_{}'.format(i)] = 'laundry'
        elif input_con[i] == '다이소':
            globals()['con_{}'.format(i)] = 'da_num'
            globals()['stand_{}'.format(i)] = 2
            globals()['table_con_{}'.format(i)] = 'daiso'
        elif input_con[i] == '실내운동시설':
            globals()['con_{}'.format(i)] = 'gym_num'
            globals()['stand_{}'.format(i)] = 4
            globals()['table_con_{}'.format(i)] = 'health'

    print(con_1)
    office = data.objects.raw(f"WITH FIRST AS( \
                        SELECT * \
                        FROM \
                            (SELECT gid,address, rent, deposit::integer, pay::integer, latitude::float, \
                            longitude::float, area, criteria, recent,station_ar, transfer,\
                        {con_0},{con_1},{con_2}, \
                        (CASE WHEN {con_0} >= {stand_0} THEN 30 ELSE (30 / {stand_0} * {con_0})::float END) AS con0, \
                        (CASE WHEN {con_1} >= {stand_1} THEN 30 ELSE (30 / {stand_1} * {con_1})::float END) AS con1, \
                        (CASE WHEN {con_2} >= {stand_2} THEN 30 ELSE (30 / {stand_2} * {con_2})::float END) AS con2,  \
                        (CASE WHEN station_ar = 2 THEN 50 \
                                WHEN station_ar = 1 THEN 30 ELSE 10 END) as st1,\
                        (CASE WHEN transfer = 1 THEN 50 ELSE 10 END) as st2, \
                        ST_Distance(geography(ST_MakePoint(longitude, latitude)),  \
                            geography(ST_MakePoint('{job[1]}', '{job[0]}')))/1000 as km \
                            FROM {table} \
                            WHERE rent='{input_rent}' and deposit <= '{input_deposit}' \
                            and pay <= '{input_pay}' and recent = 1) as r) \
                        SELECT gid,address, rent, deposit, pay, latitude, longitude, criteria, km/{gu} as distance,\
                        ((con0 + con1 + con2 + 10)/10 + (st1 + st2)/20 + time/10) as score \
                        From (SELECT *, \
                                (CASE WHEN km/{gu} <= 10 THEN 100 \
                                    WHEN km/{gu} between 10 and 90 THEN (100 - km/{gu}) ELSE 0 END) as time\
                            FROM FIRST) as k \
                        ORDER BY score desc LIMIT 10;")

    for o in office:
        print(o.gid,o.address, o.rent, o.deposit, o.pay, o.latitude, o.longitude, o.criteria)
    return office
        
    
def condition(input_house, input_rent,input_deposit,input_pay,input_con,address):
    global table
    global data
    print('condition')
    # 집유형에 따라 해당되는 {table} 도출
    if input_house == '오피스텔':
        table = 'officetels'
        data = Officetels
    elif input_house == '빌라':
        table = 'villa'
        data = Villa
    elif input_house == '원룸/투룸':
        table = 'one_two_room'
        data = OneTwoRoom
    gu, job = latlon(address)
    return score(input_house, input_rent,input_deposit,input_con,gu, job,input_pay,table, data)
              

In [132]:
condition('오피스텔', '전세','10000','0',['편의점','마트','패스트푸드'],'서울시 동대문구 이문로 1길 21')

condition
score
mart_num
196885 서울특별시 중랑구 중화동 중랑천로14길 58 전세 10000 0 37.5945315127732 127.075917984894 37.5945315127732127.075917984894
196881 서울특별시 중랑구 중화동 중랑천로14길 58 전세 9700 0 37.5945315127732 127.075917984894 37.5945315127732127.075917984894
98958 서울특별시 동대문구 이문동 이문로30길 20 전세 5500 0 37.59649294590661 127.06122846748899 37.59649294590661127.06122846748899
189834 서울특별시 종로구 숭인동 난계로29길 31 전세 1300 0 37.57400979652421 127.021480992636 37.57400979652421127.021480992636
195931 서울특별시 중랑구 상봉동 망우로 346 전세 10000 0 37.597452478030895 127.09136963878099 37.597452478030895127.09136963878099
194992 서울특별시 중랑구 망우동 망우로60길 37 전세 10000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931
194926 서울특별시 중랑구 망우동 망우로60길 37 전세 9000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931
194931 서울특별시 중랑구 망우동 망우로60길 37 전세 5000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931
93690 서울특별시 도봉구 창동 노해로69길 15-15 전세 6500 0 37.652823807117706 127.04997658903099 37.652823807

<RawQuerySet: WITH FIRST AS(                         SELECT *                         FROM                             (SELECT gid,address, rent, deposit::integer, pay::integer, latitude::float,                             longitude::float, area, criteria, recent,station_ar, transfer,                        con_num,mart_num,fast_num,                         (CASE WHEN con_num >= 10 THEN 30 ELSE (30 / 10 * con_num)::float END) AS con0,                         (CASE WHEN mart_num >= 2 THEN 30 ELSE (30 / 2 * mart_num)::float END) AS con1,                         (CASE WHEN fast_num >= 2 THEN 30 ELSE (30 / 2 * fast_num)::float END) AS con2,                          (CASE WHEN station_ar = 2 THEN 50                                 WHEN station_ar = 1 THEN 30 ELSE 10 END) as st1,                        (CASE WHEN transfer = 1 THEN 50 ELSE 10 END) as st2,                         ST_Distance(geography(ST_MakePoint(longitude, latitude)),                              geography(ST_MakePoint('127.

In [3]:
class solution:
    global gu
    global data
    def gu(self, address):
        self.address = address
        global gu
        import psycopg2
        import requests
        from urllib.parse import urlparse
        
        conn_string = "host='localhost' dbname ='house' user='postgres' password='1111'"
        conn = psycopg2.connect(conn_string)
        cur = conn.cursor()
        
        gu = address.split(' ')[1]
        cur.execute(f"SELECT speed FROM SPEED \
                WHERE area='{gu}'") #직장이 속한 구의 대중교통 km/분 평균속도

        gu = cur.fetchall()[0][0] 
        return self.gu
    
    def latlon(self, address): # 직장의 위도 경도 계산 및 직장의 구의 속도 도출    
        self.address = address
        url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
        result = requests.get(urlparse(url).geturl(),
                              headers={"Authorization":"KakaoAK 71dc765801a475a20aec8d82f539dd62"})
        json_obj = result.json()
        global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
        
        for document in json_obj['documents']:
            # 값이 NoneType 일 경우의 오류 방지
            try:
                val = [document['y'], document['x']]
            except:
                val = [0, 0]
        return self.val # 직장의 위치, 직장의 위도 경도
    
    def __init__(self, input_house, input_rent,input_deposit,input_con,address,input_pay=0):
        self.input_house = input_house
        self.input_rent = input_rent
        self.input_deposit = input_deposit
        self.input_pay = input_pay
        self.input_con = input_con
        self.address = address
        
    def score(self,input_house, input_rent,input_deposit,input_con,gu, job,input_pay,table, data):
        print('score')
        num = len(input_con)
        for i in range(num): # 편의시설의 개수만큼 for문 진행 
            if input_con[i] == '병원':
                globals()['con_{}'.format(i)] = 'hos_num'
                globals()['stand_{}'.format(i)] = 5
                globals()['table_con_{}'.format(i)] = 'hospital'
            elif input_con[i] == '마트':
                globals()['con_{}'.format(i)] = 'mart_num'
                globals()['stand_{}'.format(i)] = 2
                globals()['table_con_{}'.format(i)] = 'market'
            elif input_con[i] == '패스트푸드':
                globals()['con_{}'.format(i)] = 'fast_num'
                globals()['stand_{}'.format(i)] = 2
                globals()['table_con_{}'.format(i)] = 'fastfood'
            elif input_con[i] == '카페':
                globals()['con_{}'.format(i)] = 'cafe_num'
                globals()['stand_{}'.format(i)] = 3
                globals()['table_con_{}'.format(i)] = 'cafe'
            elif input_con[i] == '문화시설':
                globals()['con_{}'.format(i)] = 'cul_num'
                globals()['stand_{}'.format(i)] = 5
                globals()['table_con_{}'.format(i)] = 'culture'
            elif input_con[i] == '편의점':
                globals()['con_{}'.format(i)] = 'con_num'
                globals()['stand_{}'.format(i)] = 10
                globals()['table_con_{}'.format(i)] = 'convenience'
            elif input_con[i] == '세탁소':
                globals()['con_{}'.format(i)] = 'laun_num'
                globals()['stand_{}'.format(i)] = 7
                globals()['table_con_{}'.format(i)] = 'laundry'
            elif input_con[i] == '다이소':
                globals()['con_{}'.format(i)] = 'da_num'
                globals()['stand_{}'.format(i)] = 2
                globals()['table_con_{}'.format(i)] = 'daiso'
            elif input_con[i] == '실내운동시설':
                globals()['con_{}'.format(i)] = 'gym_num'
                globals()['stand_{}'.format(i)] = 4
                globals()['table_con_{}'.format(i)] = 'health'
        
        print(con_1)
        office = data.objects.raw(f"WITH FIRST AS( \
                            SELECT * \
                            FROM \
                                (SELECT gid,address, rent, deposit::integer, pay::integer, latitude::float, \
                                longitude::float, area, criteria, recent,station_ar, transfer,\
                            {con_0},{con_1},{con_2}, \
                            (CASE WHEN {con_0} >= {stand_0} THEN 30 ELSE (30 / {stand_0} * {con_0})::float END) AS con0, \
                            (CASE WHEN {con_1} >= {stand_1} THEN 30 ELSE (30 / {stand_1} * {con_1})::float END) AS con1, \
                            (CASE WHEN {con_2} >= {stand_2} THEN 30 ELSE (30 / {stand_2} * {con_2})::float END) AS con2,  \
                            (CASE WHEN station_ar = 2 THEN 50 \
                                    WHEN station_ar = 1 THEN 30 ELSE 10 END) as st1,\
                            (CASE WHEN transfer = 1 THEN 50 ELSE 10 END) as st2, \
                            ST_Distance(geography(ST_MakePoint(longitude, latitude)),  \
                                geography(ST_MakePoint('{job[1]}', '{job[0]}')))/1000 as km \
                                FROM {table} \
                                WHERE rent='{input_rent}' and deposit <= '{input_deposit}' \
                                and pay <= '{input_pay}' and recent = 1) as r) \
                            SELECT gid,address, rent, deposit, pay, latitude, longitude, criteria, km/{gu} as distance,\
                            ((con0 + con1 + con2 + 10)/10 + (st1 + st2)/20 + time/10) as score \
                            From (SELECT *, \
                                    (CASE WHEN km/{gu} <= 10 THEN 100 \
                                        WHEN km/{gu} between 10 and 90 THEN (100 - km/{gu}) ELSE 0 END) as time\
                                FROM FIRST) as k \
                            ORDER BY score desc LIMIT 10;")
        
        for o in office:
            print(o.gid,o.address, o.rent, o.deposit, o.pay, o.latitude, o.longitude, o.criteria)
        return office
        
    
    def condition(self,input_house, input_rent,input_deposit,input_con,input_pay):
        print('condition')
        # 집유형에 따라 해당되는 {table} 도출
        if input_house == '오피스텔':
            table = 'officetels'
            data = 'Officetels'
        elif input_house == '빌라':
            table = 'villa'
            data = 'Villa'
        elif input_house == '원룸/투룸':
            table = 'one_two_room'
            data = 'OneTwoRoom'
        gu = gu(address)
        job = latlon(address)
        return score(input_house, input_rent,input_deposit,input_con,gu, job,input_pay,table, data)
              

In [113]:
kk = solution('officetels', '전세','10000',['편의점','마트','패스트푸드'],'서울시 동대문구 이문로 1길 21')

In [114]:
kk.condition('officetels', '전세','10000',['편의점','마트','패스트푸드'],'서울시 동대문구 이문로 1길 21')

condition


UnboundLocalError: local variable 'gu' referenced before assignment

In [71]:
dd = solution('officetels', '전세','10000',['편의점','마트','패스트푸드'],'서울시 동대문구 이문로 1길 21')

In [72]:
def my_custom_sql(self):
    cursor = connection.cursor()    
    cursor.execute("select id_noga \
                    from myapp_Tnogahist a \
                    inner join myapp_Tdzien b on a.dziens=b.dziens \
                    where b.dzienrok = 1234")
    row = cursor.fetchone()
    return row

TypeError: score() missing 9 required positional arguments: 'input_house', 'input_rent', 'input_deposit', 'input_con', 'gu', 'job', 'input_pay', 'table', and 'data'

In [52]:
office.columns

['gid',
 'address',
 'rent',
 'deposit',
 'pay',
 'latitude',
 'longitude',
 'criteria',
 'distance',
 'score']

In [6]:
office

<RawQuerySet: WITH FIRST AS(                         SELECT *                         FROM                             (SELECT gid,address, rent, deposit::integer, pay::integer, latitude::float,                             longitude::float, area, criteria, recent,station_ar, transfer,                        fast_num,mart_num,cafe_num,                         (CASE WHEN fast_num >= 2 THEN 30 ELSE (30 / 2 * fast_num)::float END) AS con0,                         (CASE WHEN mart_num >= 2 THEN 30 ELSE (30 / 2 * mart_num)::float END) AS con1,                         (CASE WHEN cafe_num >= 3 THEN 30 ELSE (30 / 3 * cafe_num)::float END) AS con2,                          (CASE WHEN station_ar = 2 THEN 50                                 WHEN station_ar = 1 THEN 30 ELSE 10 END) as st1,                        (CASE WHEN transfer = 1 THEN 50 ELSE 10 END) as st2,                         ST_Distance(geography(ST_MakePoint(longitude, latitude)),                              geography(ST_MakePoint('127

In [44]:
ss = Speed.objects.raw(f"SELECT criteria, con_name, con_addres, con_latitu, con_longit \
                    FROM {globals()['table_con_{}'.format(i)]};")

In [48]:
raw_sql = """SELECT * FROM office as "XK" LEFT OUTER JOIN  Cafe as "AK" ON "AK".criteria = "XK".criteria ;"""
office.objects.filter(raw_sql)

AttributeError: 'RawQuerySet' object has no attribute 'objects'

In [7]:
for o in office:
    print(o.gid,o.address, o.rent, o.deposit, o.pay, o.latitude, o.longitude, o.criteria)

196885 서울특별시 중랑구 중화동 중랑천로14길 58 전세 10000 0 37.5945315127732 127.075917984894 37.5945315127732127.075917984894
196881 서울특별시 중랑구 중화동 중랑천로14길 58 전세 9700 0 37.5945315127732 127.075917984894 37.5945315127732127.075917984894
98958 서울특별시 동대문구 이문동 이문로30길 20 전세 5500 0 37.59649294590661 127.06122846748899 37.59649294590661127.06122846748899
189834 서울특별시 종로구 숭인동 난계로29길 31 전세 1300 0 37.57400979652421 127.021480992636 37.57400979652421127.021480992636
93690 서울특별시 도봉구 창동 노해로69길 15-15 전세 6500 0 37.652823807117706 127.04997658903099 37.652823807117706127.04997658903099
195931 서울특별시 중랑구 상봉동 망우로 346 전세 10000 0 37.597452478030895 127.09136963878099 37.597452478030895127.09136963878099
194931 서울특별시 중랑구 망우동 망우로60길 37 전세 5000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931
194926 서울특별시 중랑구 망우동 망우로60길 37 전세 9000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931
194992 서울특별시 중랑구 망우동 망우로60길 37 전세 10000 0 37.5971024399369 127.094238280931 37.5971024399369127.094238280931


In [42]:
latlon(address)

(0.24069938698647905, ['37.5901684571627', '127.05156387144'])

In [15]:
import re

In [16]:
aa.strip("[]").split(",")

["'병원'", " '마트'", " '패스트푸드'"]

In [13]:
import re
ccc = "['카페']"
ccc = ccc.strip('[]').split(',')
ccc = re.findall(r'"\s*([^"]*?)\s*"', str(ccc))
ccc

["'카페'"]

In [32]:
aa = "['카페']"

In [27]:
# aa = "['병원', '마트', '패스트푸드']"
place = aa.strip("[]").split(",")
place = re.findall(r'"\s*([^"]*?)\s*"', str(place))[0][1:-1]
place

'카페'

In [33]:
place = aa.strip("[]").split(",")
place = re.findall(r'"\s*([^"]*?)\s*"', str(place))
len(place)

1

In [74]:
cc[2][1:-1]

'패스트푸드'

In [19]:
res = [i.strip('""').split(",") for i in aa] 
res

[['['],
 ["'"],
 ['병'],
 ['원'],
 ["'"],
 ['', ''],
 [' '],
 ["'"],
 ['마'],
 ['트'],
 ["'"],
 ['', ''],
 [' '],
 ["'"],
 ['패'],
 ['스'],
 ['트'],
 ['푸'],
 ['드'],
 ["'"],
 [']']]

In [45]:
list_str = aa.split()
list_str

["['병원',", "'마트',", "'패스트푸드']"]